In [ ]:
import os
import json
import time
import hashlib
import uuid
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional

WALLETS_FILE = "wallets.json"
LEDGER_FILE = "transactions.json"


def sha256_hex(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

def now_ts() -> float:
    return time.time()

def pretty_amount(n: int) -> str:
    return f"{n} COIN"


@dataclass
class Transaction:
    txid: str
    timestamp: float
    from_addr: str
    to_addr: str
    amount: int
    signature: str  # simulated signature (sha256 of privkey + core fields)

    @staticmethod
    def core_string(from_addr: str, to_addr: str, amount: int, timestamp: float) -> str:
        # canonical ordering for signing
        return f"{from_addr}|{to_addr}|{amount}|{int(timestamp)}"


class WalletStore:
    """
    Stores wallets locally as {address: private_key_hex}.
    This is insecure but fine for a demo on your machine.
    """
    def __init__(self, path=WALLETS_FILE):
        self.path = path
        self._db: Dict[str, str] = {}
        self.load()

    def load(self):
        if os.path.exists(self.path):
            with open(self.path, "r", encoding="utf-8") as f:
                self._db = json.load(f)
        else:
            self._db = {}

    def save(self):
        with open(self.path, "w", encoding="utf-8") as f:
            json.dump(self._db, f, indent=2)

    @staticmethod
    def _derive_address(private_key_hex: str) -> str:
        """
        Derive a pseudo 'public address' by hashing the private key (demo only).
        Real crypto uses ECDSA/Ed25519; we avoid extra deps here.
        """
        return "addr_" + sha256_hex(bytes.fromhex(private_key_hex))[:32]

    def create_wallet(self) -> Dict[str, str]:
        priv = os.urandom(32).hex()            # 256-bit private key (hex)
        addr = self._derive_address(priv)       # derived demo address
        self._db[addr] = priv
        self.save()
        return {"address": addr, "private_key": priv}

    def list_wallets(self) -> List[str]:
        return list(self._db.keys())

    def get_privkey(self, address: str) -> Optional[str]:
        return self._db.get(address)

    def has_address(self, address: str) -> bool:
        return address in self._db
 
class Ledger:
    """
    Simple append-only transaction ledger persisted in JSON.
    Balances are computed by summing all transactions:
      - Debit from sender (from_addr)
      - Credit to recipient (to_addr)
    Genesis 'airdrops' are allowed by using from_addr="SYSTEM"
    """
    def __init__(self, path=LEDGER_FILE):
        self.path = path
        self._txs: List[Transaction] = []
        self.load()

    def load(self):
        if os.path.exists(self.path):
            with open(self.path, "r", encoding="utf-8") as f:
                data = json.load(f)
                self._txs = [Transaction(**t) for t in data]
        else:
            self._txs = []
            # No auto-genesis; you can mint via airdrop()

    def save(self):
        with open(self.path, "w", encoding="utf-8") as f:
            json.dump([asdict(t) for t in self._txs], f, indent=2)

    def all(self) -> List[Transaction]:
        return list(self._txs)

    def balances(self) -> Dict[str, int]:
        bal: Dict[str, int] = {}
        for tx in self._txs:
            if tx.from_addr != "SYSTEM":
                bal[tx.from_addr] = bal.get(tx.from_addr, 0) - tx.amount
            bal[tx.to_addr] = bal.get(tx.to_addr, 0) + tx.amount
        return bal

    def balance_of(self, address: str) -> int:
        return self.balances().get(address, 0)

    def _sign(self, priv_hex: str, from_addr: str, to_addr: str, amount: int, ts: float) -> str:
        """
        Simulated signature = sha256(priv_key_bytes + core_string_bytes)
        In real systems: sign with private key (ECDSA/Ed25519) and verify with public key.
        """
        core = Transaction.core_string(from_addr, to_addr, amount, ts).encode()
        return sha256_hex(bytes.fromhex(priv_hex) + core)

    def _verify(self, wallets: WalletStore, tx: Transaction) -> bool:
        """
        Verification (demo): recompute signature using the *stored* private key
        of the sender (insecure in real life, fine for this simulator).
        SYSTEM mints (airdrops) skip signature.
        """
        if tx.from_addr == "SYSTEM":
            return True
        priv = wallets.get_privkey(tx.from_addr)
        if not priv:
            return False
        expected = self._sign(priv, tx.from_addr, tx.to_addr, tx.amount, tx.timestamp)
        return expected == tx.signature

    def add_transaction(self, wallets: WalletStore, from_addr: str, to_addr: str, amount: int) -> Transaction:
        # basic checks
        if amount <= 0:
            raise ValueError("Amount must be positive.")
        if from_addr != "SYSTEM" and not wallets.has_address(from_addr):
            raise ValueError("Sender address not found in local wallet store.")
        if not wallets.has_address(to_addr):
            raise ValueError("Recipient address not found in local wallet store.")
        if from_addr != "SYSTEM":
            if self.balance_of(from_addr) < amount:
                raise ValueError("Insufficient balance.")

        ts = now_ts()
        txid = uuid.uuid4().hex
        # simulate signature
        if from_addr == "SYSTEM":
            sig = "GENESIS_AIRDROP"
        else:
            priv = wallets.get_privkey(from_addr)
            sig = self._sign(priv, from_addr, to_addr, amount, ts)

        tx = Transaction(
            txid=txid,
            timestamp=ts,
            from_addr=from_addr,
            to_addr=to_addr,
            amount=amount,
            signature=sig,
        )

        # verify before append
        if not self._verify(wallets, tx):
            raise ValueError("Signature verification failed (demo).")

        self._txs.append(tx)
        self.save()
        return tx

    def airdrop(self, wallets: WalletStore, to_addr: str, amount: int) -> Transaction:
        """
        Mint coins from SYSTEM to an address.
        Useful as 'genesis' funds for demos.
        """
        return self.add_transaction(wallets, "SYSTEM", to_addr, amount)


def print_header():
    print("\n=== Crypto Wallet Simulator (Educational) ===")
    print("1) Create new wallet")
    print("2) List wallets")
    print("3) Show balance of an address")
    print("4) Airdrop (SYSTEM -> address)")
    print("5) Send coins (address -> address)")
    print("6) Show recent transactions")
    print("7) Exit")

def prompt(msg: str) -> str:
    return input(msg).strip()

def show_tx(tx: Transaction):
    t = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(tx.timestamp))
    print(f"- {tx.txid} | {t} | {tx.from_addr} -> {tx.to_addr} : {tx.amount} | sig={tx.signature[:16]}...")

def main():
    wallets = WalletStore()
    ledger = Ledger()

    # If no wallets exist yet, suggest creating one
    if not wallets.list_wallets():
        print("No wallets found. Create one to get started!")

    while True:
        print_header()
        choice = prompt("Select an option (1-7): ")

        try:
            if choice == "1":
                w = wallets.create_wallet()
                print("\nNew wallet created:")
                print(f" Address:     {w['address']}")
                print(f" Private key: {w['private_key']}")
                print(" (Store your private key safely. In this demo, it is kept in wallets.json.)")

            elif choice == "2":
                addrs = wallets.list_wallets()
                if not addrs:
                    print("No wallets yet.")
                else:
                    print("\nYour wallets:")
                    for a in addrs:
                        print(" ", a)

            elif choice == "3":
                addr = prompt("Enter address: ")
                bal = ledger.balance_of(addr)
                print(f"Balance of {addr}: {pretty_amount(bal)}")

            elif choice == "4":
                to_addr = prompt("Airdrop to address: ")
                amt = int(prompt("Amount (integer): "))
                tx = ledger.airdrop(wallets, to_addr, amt)
                print("Airdrop complete:")
                show_tx(tx)

            elif choice == "5":
                from_addr = prompt("From address: ")
                to_addr = prompt("To address: ")
                amt = int(prompt("Amount (integer): "))
                tx = ledger.add_transaction(wallets, from_addr, to_addr, amt)
                print("Transaction sent:")
                show_tx(tx)

            elif choice == "6":
                txs = ledger.all()
                if not txs:
                    print("No transactions yet.")
                else:
                    print("\nRecent transactions (newest last):")
                    for tx in txs[-20:]:
                        show_tx(tx)

            elif choice == "7":
                print("Goodbye!")
                break

            else:
                print("Invalid option. Please choose 1-7.")

        except ValueError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()


No wallets found. Create one to get started!

=== Crypto Wallet Simulator (Educational) ===
1) Create new wallet
2) List wallets
3) Show balance of an address
4) Airdrop (SYSTEM -> address)
5) Send coins (address -> address)
6) Show recent transactions
7) Exit


Select an option (1-7):  1



New wallet created:
 Address:     addr_87ea61efaa89e366715a9df86c963192
 Private key: 2dc32c0293826ca526c3ffe79f3ebfad8fa28e0bae5b8e72bbfa10b9bc06995a
 (Store your private key safely. In this demo, it is kept in wallets.json.)

=== Crypto Wallet Simulator (Educational) ===
1) Create new wallet
2) List wallets
3) Show balance of an address
4) Airdrop (SYSTEM -> address)
5) Send coins (address -> address)
6) Show recent transactions
7) Exit


Select an option (1-7):  1



New wallet created:
 Address:     addr_6a420ee290ffb6c66ac09a2ea89d67f1
 Private key: 9addc969266342fecb260d84b557e47b424a3b9e146d1b1e9f3e74aa1011c8a3
 (Store your private key safely. In this demo, it is kept in wallets.json.)

=== Crypto Wallet Simulator (Educational) ===
1) Create new wallet
2) List wallets
3) Show balance of an address
4) Airdrop (SYSTEM -> address)
5) Send coins (address -> address)
6) Show recent transactions
7) Exit


...

Transaction sent:
- c6b92c775bc84d9b9414748018750c96 | 2025-08-21 18:25:51 | addr_6a420ee290ffb6c66ac09a2ea89d67f1 -> addr_87ea61efaa89e366715a9df86c963192 : 34 | sig=850c56a7d00bb041...

=== Crypto Wallet Simulator (Educational) ===
1) Create new wallet
2) List wallets
3) Show balance of an address
4) Airdrop (SYSTEM -> address)
5) Send coins (address -> address)
6) Show recent transactions
7) Exit


Select an option (1-7):  3
Enter address:  addr_87ea61efaa89e366715a9df86c963192


Balance of addr_87ea61efaa89e366715a9df86c963192: 34 COIN

=== Crypto Wallet Simulator (Educational) ===
1) Create new wallet
2) List wallets
3) Show balance of an address
4) Airdrop (SYSTEM -> address)
5) Send coins (address -> address)
6) Show recent transactions
7) Exit


Select an option (1-7):  3
Enter address:  addr_6a420ee290ffb6c66ac09a2ea89d67f1


Balance of addr_6a420ee290ffb6c66ac09a2ea89d67f1: 42 COIN

=== Crypto Wallet Simulator (Educational) ===
1) Create new wallet
2) List wallets
3) Show balance of an address
4) Airdrop (SYSTEM -> address)
5) Send coins (address -> address)
6) Show recent transactions
7) Exit
